<a href="https://colab.research.google.com/github/ValentinaEmili/Sign_language/blob/main/Hands_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# mount google drive on colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload() # here upload 'chrome-cookies.txt'

In [ ]:
!pip install -q yt_dlp

In [15]:
import pandas as pd
import cv2
import requests
import os
from google.colab.patches import cv2_imshow
from tqdm import tqdm

In [ ]:
import yt_dlp

In [16]:
# load the JSON data
js_file = pd.read_json("/content/drive/MyDrive/NLP/WLASL_v0.3.json")
folder = "/content/drive/MyDrive/NLP/dataset/"

training_folder = folder + "train/"
test_folder = folder + "test/"
training_video = training_folder + "video/"
test_video = test_folder + "video/"
training_images = training_folder + "images/"
test_images = test_folder + "images/"
os.makedirs(training_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)
os.makedirs(training_video, exist_ok=True)
os.makedirs(test_video, exist_ok=True)
os.makedirs(training_images, exist_ok=True)
os.makedirs(test_images, exist_ok=True)

In [4]:
# sources:
# {'asl5200',     youtube.com
# 'aslbrick',     mp4
# 'asldeafined',  mp4
# 'asllex',       youtu.be
# 'aslpro',       swf
# 'aslsearch',    mp4
# 'aslsignbank',  mp4
# 'aslu',         youtube.com
# 'elementalasl', mov (like mp4 maybe?)
# 'handspeak',    mp4
# 'lillybauer',   youtube.com
# 'nabboud',      youtube.com
# 'northtexas',   youtube.com
# 'scott',        youtube.com
# 'signingsavvy', mp4
# 'signschool',   mp4
# 'spreadthesign',mp4
# 'startasl',     mp4
# 'valencia-asl'} youtube.com

In [ ]:
# download youtube videos

youtube_videos = ['asl5200', 'asllex', 'aslu', 'lillybauer', 'nabboud', 'northtexas', 'scott', 'valencia-asl']
not_downloaded = 0
tot_youtube_videos = 0
downloaded = []

for i, word in enumerate(tqdm(list(js_file['gloss']), desc='glosses')):
  for j, instance in enumerate(js_file['instances'][i]):

    video_id = js_file['instances'][i][j]['video_id']
    url = js_file['instances'][i][j]['url']
    source = js_file['instances'][i][j]['source']
    split = js_file['instances'][i][j]['split']
    filename = f"{word}_{video_id}.mp4"

    if source in youtube_videos:
      tot_youtube_videos += 1
      output_path = training_video if split == 'train' else test_video
      dest_path = os.path.join(output_path, filename)
      if 'youtu.be/' in url:
          continue

      if os.path.exists(dest_path):
        downloaded.append(video_id)
        continue

      ydl_opts = {
        'outtmpl': dest_path,
        'format': 'bestvideo+bestaudio/best',
        'merge_output_format': 'mp4',
        'quiet': True,
        'sleep_interval_requests': 2,
        'max_sleep_interval': 5,
        'cookies': 'chrome-cookies.txt'
        }
      try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
          ydl.download([url])
          downloaded.append(video_id)
          #print(video_id, 'DOWNLOADED')
      except:
        #print(video_id)
        not_downloaded += 1
        continue

In [ ]:
# download non-youtube videos (mp4 or mov)
# 9734 non-youtube videos

youtube_videos = ['asl5200', 'asllex', 'aslu', 'lillybauer', 'nabboud', 'northtexas', 'scott', 'valencia-asl']

for i, word in enumerate(tqdm(list(js_file['gloss']), desc='glosses')):
  for j, instance in enumerate(js_file['instances'][i]):

    video_id = js_file['instances'][i][j]['video_id']
    url = js_file['instances'][i][j]['url']
    source = js_file['instances'][i][j]['source']
    split = js_file['instances'][i][j]['split']
    filename = f"{word}_{video_id}.mp4"

    if source not in youtube_videos:

      output_path = training_video if split == 'train' else test_video
      dest_path = os.path.join(output_path, filename)

      # remove already downloaded but damaged videos
      if os.path.exists(dest_path):
        with open(dest_path, 'rb') as f:
                header = f.read()
                if b'<html' in header.lower() or b'403 Forbidden' in header:
                  os.remove(dest_path)

      try:
        resp = requests.get(url, stream=True)
        # do not download damaged videos
        if resp.status_code != 200:
                    continue
        # save downloaded videos
        with open(dest_path, mode='wb') as f:
            f.write(resp.content)
      except:
        continue

In [49]:
!pip install -q mediapipe
# hand module
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task
# pose module
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

In [17]:
# visualization pose estimations
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [ ]:
# pose detection

import mediapipe as mp # STEP 1: Import the necessary modules.
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

sample = training_video + "adjective_01066.mp4"
i, j = 1603, 3
cap = cv2.VideoCapture(sample)
frame_end = js_file['instances'][int(i)][int(j)]['frame_end']
frame_start = js_file['instances'][int(i)][int(j)]['frame_start']
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_end = frame_end if frame_end !=-1 else total_frames
step_frame = (frame_end - frame_start + 1) // 8
curr_frame = 0
printed_frames = 0

# STEP 2: Create an PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    break
  if curr_frame < frame_start:
    curr_frame += 1
  if frame_end != -1 and curr_frame > frame_end:
    break
  if (curr_frame - frame_start) % step_frame == 0 and printed_frames < 8:
    data = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # STEP 3: Load the input image.
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=data)
    # STEP 4: Detect hand landmarks from the input image.
    detection_result = detector.detect(image)
    # STEP 5: Process the classification result. In this case, visualize it.
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    #cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
    landmarks = detection_result.pose_landmarks[0]
    for idx, landmark in enumerate(landmarks):
      x = landmark.x # Normalized x coordinate [0.0, 1.0]
      y = landmark.y # Normalized x coordinate [0.0, 1.0]
      z = landmark.z # Relative depth
      visibility = landmark.visibility # Confidence score of the landmark visibility
    printed_frames += 1
  curr_frame += 1

# details about the landmarks https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker

In [ ]:
import mediapipe as mp # STEP 1: Import the necessary modules.
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

youtube_videos = ['asl5200', 'asllex', 'aslu', 'lillybauer', 'nabboud', 'northtexas', 'scott', 'valencia-asl']

for i, word in enumerate(tqdm(list(js_file['gloss']), desc='glosses')):
  for j, instance in enumerate(js_file['instances'][i]):
    video_id = js_file['instances'][i][j]['video_id']
    url = js_file['instances'][i][j]['url']
    source = js_file['instances'][i][j]['source']
    split = js_file['instances'][i][j]['split']
    frame_end = js_file['instances'][i][j]['frame_end']
    frame_start = js_file['instances'][i][j]['frame_start']
    filename = f"{word}_{video_id}.mp4"

    if source not in youtube_videos:
      path = training_video if split == 'train' else test_video
      sample = path + filename
      cap = cv2.VideoCapture(sample)
      total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
      frame_end = frame_end if frame_end !=-1 else total_frames
      step_frame = (frame_end - frame_start + 1) // 8
      curr_frame = 0
      printed_frames = 0

      # STEP 2: Create an PoseLandmarker object.
      base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
      options = vision.PoseLandmarkerOptions(
          base_options=base_options,
          output_segmentation_masks=True)
      detector = vision.PoseLandmarker.create_from_options(options)

      while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
          break
        if curr_frame < frame_start:
          curr_frame += 1
        if frame_end != -1 and curr_frame > frame_end:
          break
        if (curr_frame - frame_start) % step_frame == 0 and printed_frames < 8:
          data = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          # STEP 3: Load the input image.
          image = mp.Image(image_format=mp.ImageFormat.SRGB, data=data)
          # STEP 4: Detect hand landmarks from the input image.
          detection_result = detector.detect(image)
          # STEP 5: Process the classification result. In this case, visualize it.
          annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
          image_path = training_images if split == 'train' else test_images
          pose_folder = image_path + "pose/"
          os.makedirs(pose_folder, exist_ok=True)
          output_path = pose_folder + f"{word}_{video_id}_{printed_frames}.jpg"
          cv2.imwrite(output_path, cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
          printed_frames += 1
        curr_frame += 1

glosses:  10%|█         | 203/2000 [1:00:12<7:24:12, 14.83s/it]

In [88]:
# visualization hand estimation

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [ ]:
# hand detection
import mediapipe as mp # STEP 1: Import the necessary modules.
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

sample = training_video + "adjective_01066.mp4"
i, j = 1603, 3
cap = cv2.VideoCapture(sample)
frame_end = js_file['instances'][int(i)][int(j)]['frame_end']
frame_start = js_file['instances'][int(i)][int(j)]['frame_start']
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_end = frame_end if frame_end !=-1 else total_frames
step_frame = (frame_end - frame_start + 1) // 16
curr_frame = 0
printed_frames = 0

# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    break
  if curr_frame < frame_start:
    curr_frame += 1
  if frame_end != -1 and curr_frame > frame_end:
    break
  if (curr_frame - frame_start) % step_frame == 0 and printed_frames < 32:
    data = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # STEP 3: Load the input image.
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=data)
    # STEP 4: Detect hand landmarks from the input image.
    detection_result = detector.detect(image)
    # STEP 5: Process the classification result. In this case, visualize it.
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    landmarks = detection_result.hand_landmarks
    if landmarks != []:
      # print only images in which the hand(s) has(ve) been detected
      cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
      for idx, hand_landmarks in enumerate(landmarks):
        x = [landmark.x for landmark in hand_landmarks] # Normalized x coordinate [0.0, 1.0]
        y = [landmark.y for landmark in hand_landmarks] # Normalized x coordinate [0.0, 1.0]
        z = [landmark.z for landmark in hand_landmarks] # Relative depth
        visibility = [landmark.visibility for landmark in hand_landmarks] # Confidence score of the landmark visibility

    printed_frames += 1
  curr_frame += 1

# details about the landmarks https://ai.google.dev/edge/mediapipe/solutions/vision/hand_landmarker